## Reading Gill anemometer with a labJack T7

Craig Lage 03-Nov-23, with help from Elana Urbach


In [ ]:
import sys
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
# Open LabJack T7
handle = ljm.openS("T7", "wifi", "139.229.178.148") 

In [ ]:
# Open LabJack T7
handle = ljm.openS("T7", "wifi","139.229.170.111") 

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

In [ ]:
info

In [ ]:
PIN_VX = "AIN0"
PIN_VY = "AIN1"
PIN_VZ = "AIN2"
PIN_VSONIC = "AIN3"
 
sample_time = .001
 
start = datetime.now()
end = start + timedelta(seconds=60*10)
 
def readout(PIN_VX, PIN_VY, PIN_VZ, PIN_VSONIC):
    vx = ljm.eReadName(handle, PIN_VX)
    vy = ljm.eReadName(handle, PIN_VY)
    vz = ljm.eReadName(handle, PIN_VZ)
    vsonic = ljm.eReadName(handle, PIN_VSONIC)
    
    # Now convert LabJack Vs to velocities and sonic Temp
    # units are +/- 5 m/s, 0 to 5 V
    vx_ms = (vx - 2.5) * 2
    vy_ms = (vy - 2.5) * 2
    vz_ms = (vz - 2.5) * 2
     
    # sonic temperature is measured on a scale of 0 to 5 V
    # from -40 to 70 C
    tsonic = -40 + vsonic/5.0 * 110

    return (datetime.now(), vx, vy, vz, vsonic, vx_ms, vy_ms, vz_ms, tsonic)


In [ ]:
(now, vx, vy, vz, vsonic, vx_ms, vy_ms, vz_ms, tsonic) = readout(PIN_VX, PIN_VY, PIN_VZ, PIN_VSONIC)
print(f"At {now.isoformat()}, Vx={vx_ms:.3f} m/s, Vy={vy_ms:.3f} m/s, Vz={vz_ms:.3f} m/s, Tsonic={tsonic:.1f}C")
print(f"AIN0={vx:.3f}, AIN1={vy:.3f}, AIN2={vz:.3f}, AIN3={vsonic:.3f}")

In [ ]:
ljm.close(handle)

In [ ]:
ljm.closeAll()

In [ ]:
names = ["AIN0", "AIN1", "AIN2", "AIN3"]
scanRate = 100

In [ ]:
readTime = 30.0
numAddresses = len(names)
aScanList = ljm.namesToAddresses(numAddresses, names)[0]
scansPerRead = int(scanRate * readTime)

scanRate = ljm.eStreamStart(handle, scansPerRead, numAddresses, aScanList, scanRate)
ret = ljm.eStreamRead(handle)
ljm.eStreamStop(handle)
aData = ret[0]


In [ ]:
aData

In [ ]:
newData = np.resize(aData, (scansPerRead, numAddresses))

In [ ]:
newData

In [ ]:
velData = np.zeros_like(newData)

In [ ]:
newData[:,0]

In [ ]:
# Now convert LabJack Vs to velocities and sonic Temp
# units are +/- 5 m/s, 0 to 5 V
# sonic temperature is measured on a scale of 0 to 5 V
# from -40 to 70 C
velData[:,0] = (newData[:,0] - 2.5) * 2.0
velData[:,1] = (newData[:,1] - 2.5) * 2.0
velData[:,2] = (newData[:,2] - 2.5) * 2.0
velData[:,3] = -40.0 + newData[:,3] / 5.0 *110.0
velNames = ["vx_ms", "vy_ms", "vz_ms", "tsonic"]



In [ ]:
xaxis = np.linspace(0,29.99,3000)

In [ ]:
xaxis[-1]

In [ ]:
velData.shape

In [ ]:
plt.subplots_adjust(hspace=0.5)
plt.suptitle("Gill Anemometer test 20240201 11:20")
plt.subplot(2,1,1)
plt.title("Sonic Temperature")
plt.plot(xaxis, velData[:,3])
plt.xlabel("Time(sec)")
plt.ylabel("Temp(C)")
plt.subplot(2,1,2)
plt.title("Wind Speed")
for i in range(3):
    plt.plot(xaxis, velData[:,i])
plt.xlabel("Time(sec)")
plt.ylabel("Wind Speed (m/s)")
#plt.savefig("/home/craiglagegit/DATA/Gill_Test_07Oct24.png")